In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1232)
np.set_printoptions(formatter={'all':lambda x: '%.3f' % x})

In [2]:
from scipy.optimize import minimize
from scipy.stats import bernoulli, binom,uniform
from scipy.special import comb
from scipy.stats import binom, bernoulli

In [3]:
from keras.datasets import mnist

In [4]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = np.where(train_X>0.5,1,0)

In [86]:
n = 60
m = n *3
sz = 28

tol = 0.01
max_iter = 100
phi = [0.33,0.33,0.33]
elbo_old = -1000000

x = np.array([train_X[np.where(train_y==7)[0][0:n],:,:],
                train_X[np.where(train_y==9)[0][0:n],:,:],
                train_X[np.where(train_y==2)[0][0:n],:,:]])
x = x.reshape(m,sz*sz,order='C')

uni_model = uniform(loc=0.25,scale=0.5)
thetas = np.array([uni_model.rvs(size=sz*sz),
                   uni_model.rvs(size=sz*sz),
                   uni_model.rvs(size=sz*sz)])
dummy = np.sum(thetas,axis=1)
thetas = thetas/dummy[:,np.newaxis]


In [63]:
# for i in range(3):  
#     ax = plt.subplot(1,3,i+1)
#     ax.imshow(x[i,i], cmap=plt.get_cmap('gray'))
# plt.show()
# dummy = x.reshape(m,sz,sz,order='C')
# print(dummy.shape)
# plt.imshow(dummy[60,:],cmap=plt.get_cmap('gray'))

In [90]:
ll_1 =  np.log(thetas[0,:])*x + np.log(1-thetas[0,:])*(1-x) + np.log(phi[0])
ll_2 =  np.log(thetas[1,:])*x + np.log(1-thetas[1,:])*(1-x) + np.log(phi[1])
ll_3 =  np.log(thetas[2,:])*x + np.log(1-thetas[2,:])*(1-x) + np.log(phi[2])



ll_1 = np.sum(ll_1,axis=1)
ll_2 = np.sum(ll_2,axis=1)
ll_3 = np.sum(ll_3,axis=1)

# denom = np.exp(ll_1) + np.exp(ll_2) + np.exp(ll_3)

w_1 = 1/(1+ np.exp(ll_2-ll_1) + np.exp(ll_3-ll_1))
w_2 = 1/(1+ np.exp(ll_1-ll_2) + np.exp(ll_3-ll_2))
w_3 = 1/(1+ np.exp(ll_2-ll_3) + np.exp(ll_1-ll_3))
# w_1 = np.exp(ll_1)/denom
# w_2 = np.exp(ll_2)/denom
# w_3 = np.exp(ll_3)/denom



/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/1408466751.py:13: RuntimeWarning: overflow encountered in exp
  w_1 = 1/(1+ np.exp(ll_2-ll_1) + np.exp(ll_3-ll_1))
/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/1408466751.py:14: RuntimeWarning: overflow encountered in exp
  w_2 = 1/(1+ np.exp(ll_1-ll_2) + np.exp(ll_3-ll_2))


In [104]:
ll_2.shape

(180,)

In [88]:
thetas[0,:] = np.sum(w_1[:,np.newaxis] * x,axis=0)/np.sum(w_1 ,axis=0) 
thetas[1,:] = np.sum(w_2[:,np.newaxis] * x,axis=0)/np.sum(w_2 ,axis=0) 
thetas[2,:] = np.sum(w_3[:,np.newaxis] * x,axis=0)/np.sum(w_3 ,axis=0) 

thetas[0,np.where(thetas[0,:]==0.0)] += 0.001
thetas[1,np.where(thetas[1,:]==0.0)] += 0.001
thetas[2,np.where(thetas[2,:]==0.0)] += 0.001

phi[0] = np.sum(w_1)/m
phi[1] = np.sum(w_2)/m
phi[2] = np.sum(w_3)/m

In [106]:
np.exp(np.log(0.001)*800)

0.0

In [43]:
for i in range(max_iter):

    ll_1 =  np.log(thetas[0,:])*x + np.log(1-thetas[0,:])*(1-x) + np.log(phi[0])
    ll_2 =  np.log(thetas[1,:])*x + np.log(1-thetas[1,:])*(1-x) + np.log(phi[1])
    ll_3 =  np.log(thetas[2,:])*x + np.log(1-thetas[2,:])*(1-x) + np.log(phi[2])

    ll_1 = np.sum(ll_1,axis=1)
    ll_2 = np.sum(ll_2,axis=1)
    ll_3 = np.sum(ll_3,axis=1)


    # ll_1[np.isnan(ll_1)]=-100
    # ll_2[np.isnan(ll_2)]=-100
    # ll_3[np.isnan(ll_3)]=-100

    denom = np.exp(ll_1) + np.exp(ll_2) + np.exp(ll_3)

    w_1 = 1/(1+ np.exp(ll_2-ll_1) + np.exp(ll_3-ll_1))
    w_2 = 1/(1+ np.exp(ll_1-ll_2) + np.exp(ll_3-ll_2))
    w_3 = 1/(1+ np.exp(ll_2-ll_3) + np.exp(ll_1-ll_3))
    # w_1 = np.exp(ll_1)/denom
    # w_2 = np.exp(ll_2)/denom
    # w_3 = np.exp(ll_3)/denom

    # elbo = np.sum(w_1 * ll_1 + w_2 * ll_2 + w_3 * ll_3)

    thetas[0,:] = np.sum(w_1[:,np.newaxis] * x,axis=0)/np.sum(w_1 ,axis=0) + 0.0001
    thetas[1,:] = np.sum(w_2[:,np.newaxis] * x,axis=0)/np.sum(w_2 ,axis=0) + 0.0001
    thetas[2,:] = np.sum(w_3[:,np.newaxis] * x,axis=0)/np.sum(w_3,axis=0) + 0.0001


    phi[0] = np.sum(w_1)/m
    phi[1] = np.sum(w_2)/m
    phi[2] = np.sum(w_3)/m

    print(f"Iteration: {i}")
    print(f"ELBO is: {elbo}")
    if np.abs(elbo-elbo_old) < tol:
        break
    elbo_old = elbo

Iteration: 0
ELBO is: -332147.8332625236
Iteration: 1
ELBO is: -106661.47241093178
Iteration: 2
ELBO is: nan
Iteration: 3
ELBO is: nan
Iteration: 4
ELBO is: nan
Iteration: 5
ELBO is: nan
Iteration: 6
ELBO is: nan
Iteration: 7
ELBO is: nan
Iteration: 8
ELBO is: nan
Iteration: 9
ELBO is: nan
Iteration: 10
ELBO is: nan
Iteration: 11
ELBO is: nan
Iteration: 12
ELBO is: nan
Iteration: 13
ELBO is: nan
Iteration: 14
ELBO is: nan
Iteration: 15
ELBO is: nan
Iteration: 16
ELBO is: nan
Iteration: 17
ELBO is: nan
Iteration: 18
ELBO is: nan
Iteration: 19
ELBO is: nan
Iteration: 20
ELBO is: nan
Iteration: 21
ELBO is: nan
Iteration: 22
ELBO is: nan
Iteration: 23
ELBO is: nan
Iteration: 24
ELBO is: nan
Iteration: 25
ELBO is: nan
Iteration: 26
ELBO is: nan
Iteration: 27
ELBO is: nan
Iteration: 28
ELBO is: nan
Iteration: 29
ELBO is: nan
Iteration: 30
ELBO is: nan
Iteration: 31
ELBO is: nan
Iteration: 32
ELBO is: nan
Iteration: 33
ELBO is: nan
Iteration: 34
ELBO is: nan
Iteration: 35
ELBO is: nan
Iterati

/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/3821970397.py:18: RuntimeWarning: overflow encountered in exp
  w_1 = 1/(1+ np.exp(ll_2-ll_1) + np.exp(ll_3-ll_1))
/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/3821970397.py:20: RuntimeWarning: overflow encountered in exp
  w_3 = 1/(1+ np.exp(ll_2-ll_3) + np.exp(ll_1-ll_3))
/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/3821970397.py:27: RuntimeWarning: invalid value encountered in divide
  thetas[0,:] = np.sum(w_1[:,np.newaxis] * x,axis=0)/np.sum(w_1 ,axis=0) + 0.0001
/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/3821970397.py:29: RuntimeWarning: invalid value encountered in divide
  thetas[2,:] = np.sum(w_3[:,np.newaxis] * x,axis=0)/np.sum(w_3,axis=0) + 0.0001
/var/folders/7n/2hstn0kx2mvgvjzthjfyy7280000gn/T/ipykernel_30945/3821970397.py:3: RuntimeWarning: divide by zero encountered in log
  ll_1 =  np.log(thetas[0,:])*x + np.log(1-thetas[0,:])*(1-x) + np.log(phi[

In [ ]:
test = thetas.reshape(3,28,28)
plt.imshow(test[0,:],cmap=plt.get_cmap('gray'))

In [ ]:
test = np.where(ll_1==100)
plt.imshow(test)

In [ ]:
thetas[0]

In [ ]:
w_2